In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('train.csv')
data.insert(0, 'Ones', 1)


In [74]:
data = data.drop(data.columns[[4, 8, 9, 11]], axis=1)

In [75]:
data = data.replace({'male': 1,'female': 0})
data = data.replace({'C': 1, 'Q': 2, 'S': 3})
data.iloc[:,3] = data.iloc[:,3].replace({3: 1,1: 3})
data.head(5)

,Ones,PassengerId,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,1,1,0,1,1,22.0,1,7.2500,3.0
1,1,2,1,3,0,38.0,1,71.2833,1.0
2,1,3,1,1,0,26.0,0,7.9250,3.0
3,1,4,1,3,0,35.0,1,53.1000,3.0
4,1,5,0,1,1,35.0,0,8.0500,3.0


In [76]:
data = data[['Ones', 'Sex', 'Pclass', 'SibSp', 'Age', 'Fare', 'Embarked', 'Survived']]
data.head(7)


,Ones,Sex,Pclass,SibSp,Age,Fare,Embarked,Survived
0,1,1,1,1,22.0,7.2500,3.0,0
1,1,0,3,1,38.0,71.2833,1.0,1
2,1,0,1,0,26.0,7.9250,3.0,1
3,1,0,3,1,35.0,53.1000,3.0,1
4,1,1,1,0,35.0,8.0500,3.0,0
5,1,1,1,0,NaN,8.4583,2.0,0
6,1,1,3,0,54.0,51.8625,3.0,0


In [77]:
cols = data.shape[1]

X = data.iloc[:,:cols-1]
Y = data.iloc[:,cols-1:cols]


X = np.matrix(X)
Y = np.matrix(Y)


X.shape, Y.shape

((891, 7), (891, 1))

In [78]:
X[:,4][np.isnan(X[:,4])] = X[:,4][np.invert(np.isnan(X[:,4]))].mean()
X[np.isnan(X)] = 1

X[:,4] = (X[:,4] - X[:,4].mean()) / X[:,4].std()
X[:,5] = (X[:,5] - X[:,5].mean()) / X[:,5].std()
X[:,6] = np.power(X[:,6], 0.5)
X[:,2] = np.power(X[:,2], 2)

In [79]:
temp = X.copy()
temp.shape

(891, 7)

In [80]:
SibClass = np.squeeze(np.asarray(np.multiply(X[:,2], X[:,3])))
SibClass.shape

(891,)

In [81]:
X = np.insert(temp, 7,SibClass, axis=1)
X[:,7] = (X[:,7] - X[:,7].mean()) / X[:,7].std()

In [82]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
Y_onehot = encoder.fit_transform(Y)
Y_onehot.shape


(891, 2)

In [83]:
def sigmoid(z):
    
    return 1/(1 + np.exp(-z))


In [84]:
def forward_prop(X, theta1, theta2):
    
    X = np.matrix(X)
    theta1 = np.matrix(theta1)
    theta2 = np.matrix(theta2)
    m = X.shape[0]
    
    a1 = X
    z2 = a1 * (theta1.T)
    a2 = np.insert(sigmoid(z2), 0, np.ones(m), axis=1)
    z3 = a2 * (theta2.T)
    hx = sigmoid(z3)
    
    return a1, z2, a2, z3, hx
    


In [85]:
def cost(params, X, Y, input_size, hidden_size, num_labels, learning_rate):
    
    X = np.matrix(X)
    Y = np.matrix(Y)
    m = X.shape[0]
    
    theta1 = np.matrix(np.reshape(params[:((input_size + 1) * hidden_size)], (hidden_size, input_size + 1)))
    theta2 = np.matrix(np.reshape(params[(input_size + 1) * hidden_size:], (num_labels, hidden_size + 1)))
    
    a1, z2, a2, z3, hx = forward_prop(X, theta1, theta2)
    
    J = 0
    
    for i in range(m):
        
        sum1 = -(Y[i,:]) * (np.log(hx[i,:]).T)
        sum2 = (1 - Y[i,:]) * (np.log(1 - hx[i,:])).T
        
        J += np.sum(sum1 - sum2)
        
    J = J/m
    
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J
           


In [94]:
input_size = 7
hidden_size = 14
num_labels = 2
learning_rate = 1


In [95]:
params = (np.random.random(size = hidden_size*(input_size + 1) + num_labels*(hidden_size + 1)) - 0.5)*0.25
cost(params, X, Y_onehot, input_size, hidden_size, num_labels, learning_rate), params.shape


(1.3755097891577124, (142,))

In [96]:
def sigmoid_gradient(z):
    
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))


In [97]:
def backprop(params, input_size, hidden_size, num_labels, X, Y, learning_rate):
    X = np.matrix(X)
    Y = np.matrix(Y)
    m = X.shape[0]
    
    theta1 = np.matrix(np.reshape(params[:(input_size + 1)*hidden_size], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size*(input_size + 1):], (num_labels, (hidden_size +1))))
    
    a1, z2, a2, z3 ,hx = forward_prop(X, theta1, theta2)
    
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    
    J = 0
    for i in range(m):
        sum1 = np.multiply(-Y[i,:], np.log(hx[i,:]))
        sum2 = np.multiply((1-Y[i,:]), np.log(1 - hx[i,:]))
        
        J += np.sum(sum1 - sum2)
        
    J = J/m
    
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    for t in range(m):
        a1t = a1[t,:]
        z2t = z2[t,:]
        a2t = a2[t,:]
        hxt = hx[t,:]
        Yt  = Y[t,:]
        
        d3t = hxt - Yt
        
        z2t = np.insert(z2t, 0, values=np.ones(1))
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))
        
        delta1 = delta1 + d2t[:,1:].T * a1t
        delta2 = delta2 + d3t.T * a2t
        
        delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
        delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
        
    delta1 = delta1 / m
    delta2 = delta2 / m

        
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad
           


In [98]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, Y_onehot, learning_rate)
J, grad.shape


(1.3755097891577124, (142,))

In [99]:
from scipy.optimize import minimize
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, Y_onehot, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 350})
fmin

     fun: 0.8433233249811839
     jac: array([  3.77632406e-05,  -6.96401979e-05,  -2.77343197e-04,
        -2.36227778e-05,   8.66762901e-06,   1.78390283e-05,
        -1.74297604e-04,  -9.72462240e-05,   9.33761534e-05,
        -1.30473452e-05,   4.90982539e-04,   1.14305372e-04,
        -3.89349723e-05,   5.55094784e-05,   1.02630248e-04,
        -1.50445898e-05,  -1.02621596e-04,  -6.19867319e-05,
        -3.09591067e-04,  -1.26984803e-04,   1.10355563e-04,
        -2.50849972e-05,   4.69570013e-05,  -2.84420698e-06,
        -1.20631947e-05,  -5.86145159e-05,   4.39067232e-05,
         4.04660720e-05,  -4.76021582e-05,   4.24680656e-05,
        -5.05685216e-05,   1.35739472e-05,  -1.02273647e-04,
        -2.52765706e-05,  -5.33872548e-04,  -1.23293849e-04,
         8.50137394e-05,  -8.49645889e-05,  -8.33623403e-05,
         1.41053026e-05,  -1.88417502e-05,  -1.05945779e-04,
        -1.48894551e-04,   9.15744208e-05,   9.18414387e-05,
         8.71907950e-05,  -2.77861620e-05,  -1

In [100]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_prop(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1))

In [117]:
np.sum(h, axis=1)

matrix([[ 1.00004348],
        [ 1.00079219],
        [ 1.00020982],
        [ 1.00020045],
        [ 1.00009414],
        [ 1.00071352],
        [ 1.00004823],
        [ 1.00016221],
        [ 1.00014953],
        [ 1.00024424],
        [ 1.00011474],
        [ 0.99969466],
        [ 1.00017139],
        [ 1.00004809],
        [ 1.0000612 ],
        [ 1.00016848],
        [ 1.00001427],
        [ 1.00047044],
        [ 1.00002211],
        [ 1.00083392],
        [ 1.00031099],
        [ 1.00084822],
        [ 1.00013684],
        [ 0.99998006],
        [ 1.00052992],
        [ 1.00001527],
        [ 1.00011979],
        [ 1.00040792],
        [ 0.9997271 ],
        [ 1.00019155],
        [ 0.99999969],
        [ 1.00028968],
        [ 1.00020002],
        [ 1.00013211],
        [ 0.99998537],
        [ 1.00058131],
        [ 1.00005199],
        [ 1.00009038],
        [ 1.00026095],
        [ 1.00048029],
        [ 1.00003764],
        [ 0.99998785],
        [ 0.99999743],
        [ 1

In [101]:
np.sum(np.fabs(Y - y_pred))


141.0

In [102]:
(891 - np.sum(np.fabs(Y - y_pred))) / 891

0.84175084175084181

In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('test.csv')
data.insert(0, 'Ones', 1)

df = data[['PassengerId']]


In [104]:
data = data.drop(data.columns[[3, 7, 8, 10]], axis=1)
data.head(3)

,Ones,PassengerId,Pclass,Sex,Age,SibSp,Fare,Embarked
0,1,892,3,male,34.5,0,7.8292,Q
1,1,893,3,female,47.0,1,7.0000,S
2,1,894,2,male,62.0,0,9.6875,Q


In [105]:
data = data[['Ones', 'Sex', 'Pclass', 'SibSp', 'Age', 'Fare', 'Embarked']]
data.head(2)

,Ones,Sex,Pclass,SibSp,Age,Fare,Embarked
0,1,male,3,0,34.5,7.8292,Q
1,1,female,3,1,47.0,7.0000,S


In [106]:
data = data.replace({'male': 1,'female': 0})
data = data.replace({'C': 1, 'Q': 2, 'S': 3})
data.iloc[:,2] = data.iloc[:,2].replace({3: 1,1: 3})
data.head(5)


,Ones,Sex,Pclass,SibSp,Age,Fare,Embarked
0,1,1,1,0,34.5,7.8292,2
1,1,0,1,1,47.0,7.0000,3
2,1,1,2,0,62.0,9.6875,2
3,1,1,1,0,27.0,8.6625,3
4,1,0,1,1,22.0,12.2875,3


In [107]:
cols = data.shape[1]

X = data.iloc[:,:cols]


X = np.matrix(X)
Y = np.matrix(Y)


X.shape

(418, 7)

In [108]:
X[:,4][np.isnan(X[:,4])] = X[:,4][np.invert(np.isnan(X[:,4]))].mean()
X[np.isnan(X)] = 1

X[:,4] = (X[:,4] - X[:,4].mean()) / X[:,4].std()
X[:,5] = (X[:,5] - X[:,5].mean()) / X[:,5].std()
X[:,6] = np.power(X[:,6], 0.5)
X[:,2] = np.power(X[:,2], 2)





In [109]:
temp = X.copy()
SibClass = np.squeeze(np.asarray(np.multiply(X[:,2], X[:,3])))

In [110]:
X = np.insert(temp, 7,SibClass, axis=1)
X[:,7] = (X[:,7] - X[:,7].mean()) / X[:,7].std()

In [111]:
X.shape

(418, 8)

In [112]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_prop(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1))


In [113]:
predictions = y_pred.astype(int)

In [114]:
df.insert(1, 'Survived', predictions)

In [115]:
df.to_csv('predictions2222.csv')